In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.5/701.5 kB 5.9 MB/s eta 0:00:00


In [ ]:

## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [ ]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/yolov8/Data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 2085 images---


  0%|          | 0/1668 [00:00<?, ?it/s]

------ Training data created with 80% split 1668 images -------


  0%|          | 0/417 [00:00<?, ?it/s]

------ Testing data created with a total of 417 images ----------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/107.7 GB disk)


In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yolov8/data.yaml epochs=100 imgsz=640 batch=8 project=/content/drive/MyDrive/yolov8/training_results name=plates

Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/yolov8/data.yaml, epochs=100, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=plates3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/plates2/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8/test_images

Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/14 /content/drive/MyDrive/yolov8/test_images/0004_jpg.rf.0231b73b7cf7d709633c0590860c9a92.jpg: 640x640 1 plate, 22.4ms
image 2/14 /content/drive/MyDrive/yolov8/test_images/0007_jpg.rf.aac5bca9738b2e6b80b14615261b15b3.jpg: 640x640 1 plate, 16.3ms
image 3/14 /content/drive/MyDrive/yolov8/test_images/0008_jpg.rf.2a7bc7b1f00025ad1ea86b765c1e1004.jpg: 640x640 1 plate, 16.2ms
image 4/14 /content/drive/MyDrive/yolov8/test_images/0009_jpg.rf.5148452ffadb79377def82aa48e7b7ab.jpg: 640x640 1 plate, 16.2ms
image 5/14 /content/drive/MyDrive/yolov8/test_images/0012_jpg.rf.7521797aed0c16df17478fb2b52245de.jpg: 640x640 1 plate, 16.2ms
image 6/14 /content/drive/MyDrive/yolov8/test_images/0017_jpg.rf.45241f6bd0b1b54c298f5e3ba1a402e4.jpg: 640x640 1 plate, 16.2ms
image 7/14 /content/drive/My

In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/plates2/weights/best.pt conf=0.77 source=/content/drive/MyDrive/yolov8/vid2

Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (1/478) /content/drive/MyDrive/yolov8/vid2/Download (1).mp4: 640x384 1 plate, 110.4ms
video 1/1 (2/478) /content/drive/MyDrive/yolov8/vid2/Download (1).mp4: 640x384 1 plate, 10.7ms
video 1/1 (3/478) /content/drive/MyDrive/yolov8/vid2/Download (1).mp4: 640x384 1 plate, 10.7ms
video 1/1 (4/478) /content/drive/MyDrive/yolov8/vid2/Download (1).mp4: 640x384 1 plate, 10.6ms
video 1/1 (5/478) /content/drive/MyDrive/yolov8/vid2/Download (1).mp4: 640x384 1 plate, 10.6ms
video 1/1 (6/478) /content/drive/MyDrive/yolov8/vid2/Download (1).mp4: 640x384 1 plate, 10.6ms
video 1/1 (7/478) /content/drive/MyDrive/yolov8/vid2/Download (1).mp4: 640x384 1 plate, 10.6ms
video 1/1 (8/478) /content/drive/MyDrive/yolov8/vid2/Download (1).mp4: 640x384 1 plate, 10.6ms
video 1/1 (9/478) /content/drive/MyDrive/yolov8/vid2/Download (1).mp4: 

In [ ]:
cp -r /content/runs/detect/predict2 /content/drive/MyDrive/yolov8/output

In [ ]:
!yolo mode=export model=/content/drive/MyDrive/yolov8/training_results/plates2/weights/best.pt format=onnx

Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/yolov8/training_results/plates2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 85.4 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 11.6s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 13.3s, saved as '/content/drive/MyDrive/yolov8/training_results/plates2/weights/best.onnx' (42.6 MB)

Export complete (16.3s)
Results saved to /content/drive/MyDrive/yolov8/training_results/plates2/weights
Predict:         yolo predict task=detect mod